In [1]:
#Import libraries and create pass to download data
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from numpy.linalg import inv
import wrds
db=wrds.Connection(wrds_username='michdan')

Loading library list...
Done


### Daily Data

In [2]:
#import data from CRSP
stock_daily_data = db.raw_sql("""
                    select a.permno, a.date, a.ret
                      from crsp.dsf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1925' and '12/31/2022'
                      and b.exchcd between 1 and 3
                      and b.shrcd between 10 and 11
                      """, date_cols=['date']) 

#import data from CRSP
Rmd=db.raw_sql("""select  date,vwretd from crsp.dsi 
                where date>='1925-01-01' and date<='2022-12-31'
                """,date_cols=['date'])

#modify data
stock_daily_data = stock_daily_data.rename(columns={'ret':'Rn'})
Rmd = Rmd.rename(columns={'vwretd':'Rmd'})

#save data in local
stock_daily_data.to_csv('data_daily')
Rmd.to_csv('Rm_daily.csv')

### Monthly Data

In [ ]:
#import data from CRSP
stock_data = db.raw_sql("""
                    select a.permno, a.date, a.ret, a.shrout, a.prc, b.siccd
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1925' and '12/31/2022'
                      and b.exchcd between 1 and 3
                      and b.shrcd between 10 and 11
                      """, date_cols=['date']) 

#import data from CRSP
Rf=db.raw_sql("""select  mcaldt,tmytm 
           from crsp.tfz_mth_rf            
            where kytreasnox = 2000001 
           and mcaldt>='1925-01-01'
            and mcaldt<='2022-12-31'""", date_cols=['mcaldt'])

#import data from CRSP
Rm=db.raw_sql("""select  date,vwretd from crsp.msi 
                where date>='1925-01-01' and date<='2022-12-31'
                """,date_cols=['date'])

#modify data
stock_data = stock_data.rename(columns={'ret':'Rn'})

Rf['tmytm']=np.exp(Rf['tmytm']/12/100)-1
Rf=Rf.rename(columns={ "mcaldt": "date","tmytm": "rf"})

Rm = Rm.rename(columns={'vwretd':'Rm'})

data = pd.merge(stock_data, Rf, on=['date'], how='left')
data = pd.merge(data, Rm, on=['date'], how='left')
data = data.sort_values(['permno','date'])
data['mcap'] = np.abs(data['prc'])*data['shrout']
data['mcap_l'] = data.groupby(['permno'])['mcap'].shift(1)
data['Rn_f'] = data.groupby(['permno'])['Rn'].shift(-1)
data['const'] = 1
data['Rn_e'] = data['Rn']-data['rf']
data['Rm_e'] = data['Rm']-data['rf']
data['w_m'] = data['mcap_l']/data.groupby(['date'])['mcap_l'].transform('sum')
data = data.dropna(subset=['mcap_l','Rn_e','Rm_e']).copy()

#save data in local
data.to_csv('data_monthly.csv')
Rm.to_csv('Rm.csv')
Rf.to_csv('Rf.csv')